# Few-Shot Templates
## Overview

In this notebook, we dive into **Few-Shot Prompting**, a powerful technique that allows language models to generate high-quality outputs with minimal fine-tuning. By providing carefully selected examples, Few-Shot Prompting ensures precise and context-aware responses across a variety of use cases.
## What is Few-Shot Prompting?

**Few-Shot Prompting** is a technique that guides language models to generate high-quality responses by providing a small number of carefully chosen examples. Unlike traditional methods, which may require extensive fine-tuning, few-shot prompting leverages the model’s inherent ability to generalize from examples.

### Key Concepts:
- **Example-Driven**: Few-shot prompting relies on providing a few relevant examples within the prompt.
- **Context-Aware**: The provided examples help the model understand the desired context, making it more likely to generate accurate and context-sensitive outputs.
- **Reduced Fine-Tuning**: This approach minimizes the need for expensive and time-consuming fine-tuning of the model.


In [1]:
%%capture --no-stderr
%pip install langchain_community langchain langchain_nvidia_ai_endpoints langchain-chroma

In [2]:
import getpass
import os

# Set up your api key
if not os.environ.get("NVIDIA_API_KEY"):
  os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter API key for NVIDIA: ")

Enter API key for NVIDIA: ··········


In [3]:
from langchain.chat_models import init_chat_model
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

# Select the chat model
llm = init_chat_model('meta/llama-3.1-70b-instruct', model_provider = 'nvidia')

# Select the embedding model
embeddings = NVIDIAEmbeddings(model='NV-Embed-QA')

response = llm.predict("Hello, how are you?")
print(response)

<ipython-input-3-75c92557114f>:10: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict("Hello, how are you?")


Hello! I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to chat with you! How can I assist you today?


In [4]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

prompt = PromptTemplate.from_template("Question: {question}\n{answer}")


few_shot_prompt = [
    {
    "question": "Did Leonardo da Vinci and Isaac Newton live in the same century?",
    "answer": """
Are follow up questions needed here: Yes.
Follow up: When was Leonardo da Vinci born and when did he die?
Intermediate answer: Leonardo da Vinci was born in 1452 and died in 1519.
Follow up: When was Isaac Newton born and when did he die?
Intermediate answer: Isaac Newton was born in 1643 and died in 1727.
So the final answer is: No
"""
},
    {
    "question": "Who has more Grammy Awards, Taylor Swift or Beyoncé?",
    "answer": """
Are follow up questions needed here: Yes.
Follow up: How many Grammy Awards has Taylor Swift won?
Intermediate answer: Taylor Swift has won 14 Grammy Awards (as of 2024).
Follow up: How many Grammy Awards has Beyoncé won?
Intermediate answer: Beyoncé has won 32 Grammy Awards (as of 2024).
So the final answer is: Beyoncé
"""
},
{
    "question": "Which came first, the invention of the telephone or the light bulb?",
    "answer": """
Are follow up questions needed here: Yes.
Follow up: When was the telephone invented?
Intermediate answer: The telephone was invented by Alexander Graham Bell in 1876.
Follow up: When was the light bulb invented?
Intermediate answer: Thomas Edison developed a practical incandescent light bulb in 1879.
So the final answer is: The telephone
"""
},
{
    "question": "Was the capital of Brazil always Brasília?",
    "answer": """
Are follow up questions needed here: Yes.
Follow up: What is the current capital of Brazil?
Intermediate answer: The current capital of Brazil is Brasília.
Follow up: Was there a different capital before Brasília?
Intermediate answer: Yes, the capital was Rio de Janeiro before Brasília became the capital in 1960.
So the final answer is: No
"""
}
]


In [5]:
print(prompt.invoke(few_shot_prompt[0]).to_string())

Question: Did Leonardo da Vinci and Isaac Newton live in the same century?

Are follow up questions needed here: Yes.
Follow up: When was Leonardo da Vinci born and when did he die?
Intermediate answer: Leonardo da Vinci was born in 1452 and died in 1519.
Follow up: When was Isaac Newton born and when did he die?
Intermediate answer: Isaac Newton was born in 1643 and died in 1727.
So the final answer is: No



In [6]:
# PAssing the prompt to FEwPromptTemplate
few_shot_prompt_template = FewShotPromptTemplate(
    examples=few_shot_prompt,
    example_prompt=prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
    example_separator="\n\n"
)

# Change 'input' to 'question' in the invoke call
print(few_shot_prompt_template.invoke({'question': 'when did the dinousors go extinct?'}).to_string())

Question: Did Leonardo da Vinci and Isaac Newton live in the same century?

Are follow up questions needed here: Yes.
Follow up: When was Leonardo da Vinci born and when did he die?
Intermediate answer: Leonardo da Vinci was born in 1452 and died in 1519.
Follow up: When was Isaac Newton born and when did he die?
Intermediate answer: Isaac Newton was born in 1643 and died in 1727.
So the final answer is: No


Question: Who has more Grammy Awards, Taylor Swift or Beyoncé?

Are follow up questions needed here: Yes.
Follow up: How many Grammy Awards has Taylor Swift won?
Intermediate answer: Taylor Swift has won 14 Grammy Awards (as of 2024).
Follow up: How many Grammy Awards has Beyoncé won?
Intermediate answer: Beyoncé has won 32 Grammy Awards (as of 2024).
So the final answer is: Beyoncé


Question: Which came first, the invention of the telephone or the light bulb?

Are follow up questions needed here: Yes.
Follow up: When was the telephone invented?
Intermediate answer: The telephone

In [7]:
respnse = llm.predict(few_shot_prompt_template.invoke({'question': 'when did the dinousors go extinct?'}).to_string())
print(respnse)

It seems like you're asking a question that requires additional information, just like the previous questions.

Let's go through it:

Are follow-up questions needed here: Yes.

Follow up: What is the scientifically accepted time period of the dinosaurs' extinction?
Intermediate answer: According to paleontologists, dinosaurs went extinct at the end of the Cretaceous period, about 65 million years ago.

Follow up: What event is believed to have caused the mass extinction of dinosaurs?
Intermediate answer: A massive asteroid impact, combined with other factors like intense volcanic activity and a dramatic change in the Earth's climate, is thought to have led to the extinction of the dinosaurs.

So the final answer is: About 65 million years ago.


## 2. Dynamic Example Selector
- Embed your examples.
- Store them in Chroma.
- Dynamically select the top-k most similar examples for each new question.
- Format the full prompt with helpful context for better LLM reasoning.

 breaking down complex questions into intermediate reasoning steps enhances response quality, and using semantic similarity with a vector store like Chroma is a great way to support that dynamically

In [10]:
from langchain_chroma import Chroma
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings


embeddings = NVIDIAEmbeddings()

chroma = Chroma(persist_directory="example_selector", embedding_function=embeddings)

few_shot_prompt = [
    {
    "question": "Did Leonardo da Vinci and Isaac Newton live in the same century?",
    "answer": """
Are follow up questions needed here: Yes.
Follow up: When was Leonardo da Vinci born and when did he die?
Intermediate answer: Leonardo da Vinci was born in 1452 and died in 1519.
Follow up: When was Isaac Newton born and when did he die?
Intermediate answer: Isaac Newton was born in 1643 and died in 1727.
So the final answer is: No
"""
},
    {
    "question": "Who has more Grammy Awards, Taylor Swift or Beyoncé?",
    "answer": """
Are follow up questions needed here: Yes.
Follow up: How many Grammy Awards has Taylor Swift won?
Intermediate answer: Taylor Swift has won 14 Grammy Awards (as of 2024).
Follow up: How many Grammy Awards has Beyoncé won?
Intermediate answer: Beyoncé has won 32 Grammy Awards (as of 2024).
So the final answer is: Beyoncé
"""
},
{
    "question": "Which came first, the invention of the telephone or the light bulb?",
    "answer": """
Are follow up questions needed here: Yes.
Follow up: When was the telephone invented?
Intermediate answer: The telephone was invented by Alexander Graham Bell in 1876.
Follow up: When was the light bulb invented?
Intermediate answer: Thomas Edison developed a practical incandescent light bulb in 1879.
So the final answer is: The telephone
"""
},
{
    "question": "Was the capital of Brazil always Brasília?",
    "answer": """
Are follow up questions needed here: Yes.
Follow up: What is the current capital of Brazil?
Intermediate answer: The current capital of Brazil is Brasília.
Follow up: Was there a different capital before Brasília?
Intermediate answer: Yes, the capital was Rio de Janeiro before Brasília became the capital in 1960.
So the final answer is: No
"""
}
]



In [14]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    few_shot_prompt,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    NVIDIAEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=1,
)

# Select the most similar example to the input.
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

Examples most similar to the input: Who was the father of Mary Ball Washington?


answer: 
Are follow up questions needed here: Yes.
Follow up: When was the telephone invented?
Intermediate answer: The telephone was invented by Alexander Graham Bell in 1876.
Follow up: When was the light bulb invented?
Intermediate answer: Thomas Edison developed a practical incandescent light bulb in 1879.
So the final answer is: The telephone

question: Which came first, the invention of the telephone or the light bulb?


In [16]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(
    prompt.invoke({"input": "Who was the father of Mary Ball Washington?"}).to_string()
)

Question: Which came first, the invention of the telephone or the light bulb?

Are follow up questions needed here: Yes.
Follow up: When was the telephone invented?
Intermediate answer: The telephone was invented by Alexander Graham Bell in 1876.
Follow up: When was the light bulb invented?
Intermediate answer: Thomas Edison developed a practical incandescent light bulb in 1879.
So the final answer is: The telephone


Question: Who was the father of Mary Ball Washington?


## 3. FewShotChatMessagePromptTemplate
This specific template is designed for chat-based models (like GPT-4 in chat mode). Instead of producing a plain text string, it builds a series of chat messages that simulate a real conversation:

- Each example is shown as a conversation between a "human" and an "AI".

- After the few-shot examples, the user’s actual message is appended.

- The model sees all of this context before it responds.

This structure mimics a real dialogue and is especially useful when you're aiming for a natural, conversational tone.

In [17]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Initialize OpenAI embeddings
embeddings = NVIDIAEmbeddings()

# Initialize the Vector DB
chroma = Chroma(persist_directory="fewshot_chat", embedding_function=embeddings)


# Define examples for few-shot prompting :
examples = [
    {
        "instruction": "You are an expert in writing meeting minutes. Please write meeting minutes based on the given information",
        "input": "On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader John Smith, Digital Marketing Manager Sarah Johnson, and Social Media Manager Mike Wilson. The main purpose of the meeting was to establish marketing strategies for the first half of 2024 and discuss ideas for new social media campaigns. Team Leader John Smith provided a brief overview of recent market trends, followed by presentations from each team member about strategic ideas in their respective fields.",
        "answer": """
Meeting Minutes: XYZ Company Marketing Strategy Meeting
Date: December 25, 2023
Location: XYZ Company Conference Room
Attendees: John Smith (Marketing Team Leader), Sarah Johnson (Digital Marketing Manager), Mike Wilson (Social Media Manager)

1. Opening
   - Meeting began with opening remarks by Team Leader John Smith.
   - Meeting purpose: Establish marketing strategies for H1 2024 and discuss new social media campaign ideas.

2. Market Trend Overview (John Smith)
   - Team Leader John Smith presented analysis of recent market trends.
   - Shared insights on consumer behavior changes and competitor strategies.
3. Digital Marketing Strategy (Sarah Johnson)
   - Sarah Johnson presented digital marketing strategy.
   - Focused on online advertising and SEO optimization methods.

4. Social Media Campaign (Mike Wilson)
   - Mike Wilson proposed ideas for new social media campaigns.
   - Explained plans for influencer marketing and content strategy.

5. Comprehensive Discussion
   - Team members shared and discussed ideas.
   - Discussed budget and resource allocation for each strategy.

6. Closing
   - Confirmed next meeting date and time.
   - Sarah Johnson assigned to compile and distribute meeting minutes.
""",
    },
    {
        "instruction": "You are a summarization expert. Please summarize the content based on the given information",
        "input": "This document is a 20-page report on 'Strategies for Sustainable Urban Development'. The report comprehensively covers the importance of sustainable urban development, current urbanization issues, and various strategies to make urban development sustainable. The report also introduces successful sustainable urban development cases from multiple countries and summarizes the lessons learned from these cases.",
        "answer": """
Document Summary: Strategy Report for Sustainable Urban Development

- Importance: Emphasizes the necessity of sustainable urban development and its social, economic, and environmental benefits.
- Current Issues: Analyzes major problems in current urbanization processes, such as environmental pollution, resource depletion, and increasing inequality.
- Strategies: Presents various strategies to achieve sustainable urban development. These include eco-friendly construction, public transportation improvements, energy efficiency enhancement, and strengthening community engagement.
- Case Studies: Introduces successful sustainable development cases from cities worldwide. For example, explains achievable strategies through cases like Copenhagen, Denmark and Yokohama, Japan.
- Lessons: Summarizes key lessons learned from these cases. Emphasized lessons include the importance of multi-faceted approaches, cooperation with local communities, and the need for long-term planning.

This report provides an in-depth analysis of how sustainable urban development can be realized in practical and effective forms.
""",
    },
    {
        "instruction": "You are a sentence correction expert. Please correct the following sentences",
        "input": "Our company is planning to introduce a new marketing strategy. Through this, communication with customers will become more effective.",
        "answer": "This company expects to improve customer communication more effectively by introducing a new marketing strategy.",
    },
]

In [18]:
# Add examples to the vector store
texts = [example["instruction"] + " " + example["input"] for example in examples]
metadatas = examples
chroma.add_texts(texts=texts, metadatas=metadatas)

# Example query
query = {
    "instruction": "Please write the meeting minutes",
    "input": "On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting was attended by Project Manager Choi Hyun-soo, Lead Developer Hwang Ji-yeon, and UI/UX Designer Kim Tae-young. The main purpose of the meeting was to review the current progress of the project and establish plans for upcoming milestones. Each team member provided updates on their respective areas of work, and the team set goals for the next week.",
}

# Perform similarity search
query_text = query["instruction"] + " " + query["input"]
results = chroma.similarity_search(query_text, k=1)
print(results)

[Document(id='f75696d8-7655-4fa7-8fd6-d1f8cafc80d7', metadata={'answer': '\nMeeting Minutes: XYZ Company Marketing Strategy Meeting\nDate: December 25, 2023\nLocation: XYZ Company Conference Room\nAttendees: John Smith (Marketing Team Leader), Sarah Johnson (Digital Marketing Manager), Mike Wilson (Social Media Manager)\n\n1. Opening\n   - Meeting began with opening remarks by Team Leader John Smith.\n   - Meeting purpose: Establish marketing strategies for H1 2024 and discuss new social media campaign ideas.\n\n2. Market Trend Overview (John Smith)\n   - Team Leader John Smith presented analysis of recent market trends.\n   - Shared insights on consumer behavior changes and competitor strategies.\n3. Digital Marketing Strategy (Sarah Johnson)\n   - Sarah Johnson presented digital marketing strategy.\n   - Focused on online advertising and SEO optimization methods.\n\n4. Social Media Campaign (Mike Wilson)\n   - Mike Wilson proposed ideas for new social media campaigns.\n   - Explained

In [19]:
# Print the most similar example
print(f"Most similar example to the input:\n{query_text}\n")
for result in results:
    print(f'Instruction:\n{result.metadata["instruction"]}')
    print(f'Input:\n{result.metadata["input"]}')
    print(f'Answer:\n{result.metadata["answer"]}')

Most similar example to the input:
Please write the meeting minutes On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting was attended by Project Manager Choi Hyun-soo, Lead Developer Hwang Ji-yeon, and UI/UX Designer Kim Tae-young. The main purpose of the meeting was to review the current progress of the project and establish plans for upcoming milestones. Each team member provided updates on their respective areas of work, and the team set goals for the next week.

Instruction:
You are an expert in writing meeting minutes. Please write meeting minutes based on the given information
Input:
On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader John Smith, Digital Marketing Manager Sarah Johnson, and Social Media Manager Mike Wilson. The main purpose of the meeting was to establish marketing strategies for the f

In [20]:
# Create the final prompt template
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        (
            "human",
            "{instruction}:\n{input}",
        ),
    ]
)

# Combine the prompt and the best example
best_example = results[0].metadata
filled_prompt = final_prompt.format_messages(**best_example)

# Print the filled prompt
print("\nFilled Prompt:\n")
for message in filled_prompt:
    # Determine message type and extract content
    message_type = type(message).__name__  # e.g., SystemMessage, HumanMessage, AIMessage
    content = message.content
    print(f"{message_type}:\n{content}\n")


Filled Prompt:

SystemMessage:
You are a helpful assistant.

HumanMessage:
You are an expert in writing meeting minutes. Please write meeting minutes based on the given information:
On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader John Smith, Digital Marketing Manager Sarah Johnson, and Social Media Manager Mike Wilson. The main purpose of the meeting was to establish marketing strategies for the first half of 2024 and discuss ideas for new social media campaigns. Team Leader John Smith provided a brief overview of recent market trends, followed by presentations from each team member about strategic ideas in their respective fields.



In [21]:
# Query the model with the filled prompt
formatted_prompt = "\n".join([f"{type(message).__name__}:\n{message.content}" for message in filled_prompt])
response = llm.predict(formatted_prompt)

# Print the model's response
print("\nResponse:\n")
print(response)


Response:

**Meeting Minutes: XYZ Company Marketing Strategy Meeting**

**Date:** December 25, 2023
**Time:** 3:00 PM
**Attendees:**

* John Smith, Marketing Team Leader
* Sarah Johnson, Digital Marketing Manager
* Mike Wilson, Social Media Manager

**Objective:**
The main purpose of the meeting was to discuss and establish marketing strategies for the first half of 2024 and explore new ideas for social media campaigns.

**Summary of Discussions:**

1. **Market Trends Overview:** John Smith provided a brief overview of recent market trends, highlighting key areas of focus for the company's marketing efforts in the upcoming year.
2. **Strategic Presentations:** Each team member presented their ideas for strategic marketing initiatives in their respective fields:
	* Sarah Johnson shared digital marketing strategies to enhance the company's online presence and drive website traffic.
	* Mike Wilson presented social media campaign ideas to increase engagement and reach new audiences.
3. **

## 4. Dynamic few-shot Prompting
Sometimes, it’s not ideal to always show the same few-shot examples for every input. Some questions might benefit from different types of examples depending on what they’re asking. That’s where dynamic few-shot prompting comes in.

Instead of hardcoding your examples, you let the system choose examples based on how semantically similar they are to the current input. This improves relevance and overall model performance.




In [22]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

examples = [
    {"input": "2 plus 2", "output": "4"},
    {"input": "2 plus 3", "output": "5"},
    {"input": "2 plus 4", "output": "6"},
    {"input": "What did the tree say to the wind?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the stars",
        "output": "Stars above and stars below, where they end we do not know.",
    },
    {
        "input": "Tell me a joke about birds",
        "output": "Why don’t birds use social media? They already tweet.",
    },
    {
        "input": "3 plus 5",
        "output": "8"
    },
    {
        "input": "What did the rock say to the sky?",
        "output": "It just sat there in silence."
    }
]


to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = NVIDIAEmbeddings()
vectorstores = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [23]:
# create a selector
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstores,
    k=2
)

example_selector.select_examples({"input": "2 plus 2"})

[{'input': '2 plus 2', 'output': '4'}, {'input': '2 plus 3', 'output': '5'}]

In [24]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

print(few_shot_prompt.invoke(input='whats 5 plus 10').to_messages())

[HumanMessage(content='2 plus 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={}), HumanMessage(content='3 plus 5', additional_kwargs={}, response_metadata={}), AIMessage(content='8', additional_kwargs={}, response_metadata={})]


In [26]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(few_shot_prompt.invoke(input="What's 3 * 3?"))

messages=[HumanMessage(content='3 plus 5', additional_kwargs={}, response_metadata={}), AIMessage(content='8', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 plus 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={})]


In [27]:
chain = final_prompt | llm

chain.invoke({"input": "What's 4 * 3?"})

AIMessage(content='With a flick of my wand and a whisper of mathemagical incantations, I decree... 12!', additional_kwargs={}, response_metadata={'role': 'assistant', 'content': 'With a flick of my wand and a whisper of mathemagical incantations, I decree... 12!', 'token_usage': {'prompt_tokens': 62, 'total_tokens': 86, 'completion_tokens': 24}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-70b-instruct'}, id='run-08f5336c-2748-4e16-82c0-c7a35dd727ae-0', usage_metadata={'input_tokens': 62, 'output_tokens': 24, 'total_tokens': 86}, role='assistant')